# Regularization with SciKit-Learn

Previously we created a new polynomial feature set and then applied our standard linear regression on it, but we can be smarter about model choice and utilize regularization.

Regularization attempts to minimize the RSS (residual sum of squares) *and* a penalty factor. This penalty factor will penalize models that have coefficients that are too large. Some methods of regularization will actually cause non useful features to have a coefficient of zero, in which case the model does not consider the feature.

Let's explore two methods of regularization, Ridge Regression and Lasso. We'll combine these with the polynomial feature set (it wouldn't be as effective to perform regularization of a model on such a small original feature set of the original X).

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Data and Setup

In [2]:
df = pd.read_csv("Advertising.csv")
X = df.drop('sales',axis=1)
y = df['sales']

In [3]:
X.head()

,TV,radio,newspaper
0,230.1,37.8,69.2
1,44.5,39.3,45.1
2,17.2,45.9,69.3
3,151.5,41.3,58.5
4,180.8,10.8,58.4


In [4]:
y.head()

0    22.1
1    10.4
2     9.3
3    18.5
4    12.9
Name: sales, dtype: float64

### Polynomial Conversion

In [5]:
from sklearn.preprocessing import PolynomialFeatures

In [6]:
polynomial_converter = PolynomialFeatures(degree=3,include_bias=False)

In [7]:
poly_features = polynomial_converter.fit_transform(X)

In [8]:
poly_features.shape

(200, 19)

In [10]:
# poly_features
pd.DataFrame(poly_features).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,230.1,37.8,69.2,52946.01,8697.78,15922.92,1428.84,2615.76,4788.64,1.218288e+07,2001359.178,3663863.892,328776.084,601886.376,1101866.064,54010.152,98875.728,181010.592,331373.888
1,44.5,39.3,45.1,1980.25,1748.85,2006.95,1544.49,1772.43,2034.01,8.812112e+04,77823.825,89309.275,68729.805,78873.135,90513.445,60698.457,69656.499,79936.593,91733.851
2,17.2,45.9,69.3,295.84,789.48,1191.96,2106.81,3180.87,4802.49,5.088448e+03,13579.056,20501.712,36237.132,54710.964,82602.828,96702.579,146001.933,220434.291,332812.557
3,151.5,41.3,58.5,22952.25,6256.95,8862.75,1705.69,2416.05,3422.25,3.477266e+06,947927.925,1342706.625,258412.035,366031.575,518470.875,70444.997,99782.865,141338.925,200201.625
4,180.8,10.8,58.4,32688.64,1952.64,10558.72,116.64,630.72,3410.56,5.910106e+06,353037.312,1909016.576,21088.512,114034.176,616629.248,1259.712,6811.776,36834.048,199176.704


In [11]:
polynomial_converter.get_feature_names_out()

array(['TV', 'radio', 'newspaper', 'TV^2', 'TV radio', 'TV newspaper',
       'radio^2', 'radio newspaper', 'newspaper^2', 'TV^3', 'TV^2 radio',
       'TV^2 newspaper', 'TV radio^2', 'TV radio newspaper',
       'TV newspaper^2', 'radio^3', 'radio^2 newspaper',
       'radio newspaper^2', 'newspaper^3'], dtype=object)

In [12]:
poly_feature_names = list(polynomial_converter.get_feature_names_out())
poly_features_df = pd.DataFrame(poly_features,columns=poly_feature_names)
poly_features_df.head()

,TV,radio,newspaper,TV^2,TV radio,TV newspaper,radio^2,radio newspaper,newspaper^2,TV^3,TV^2 radio,TV^2 newspaper,TV radio^2,TV radio newspaper,TV newspaper^2,radio^3,radio^2 newspaper,radio newspaper^2,newspaper^3
0,230.1,37.8,69.2,52946.01,8697.78,15922.92,1428.84,2615.76,4788.64,1.218288e+07,2001359.178,3663863.892,328776.084,601886.376,1101866.064,54010.152,98875.728,181010.592,331373.888
1,44.5,39.3,45.1,1980.25,1748.85,2006.95,1544.49,1772.43,2034.01,8.812112e+04,77823.825,89309.275,68729.805,78873.135,90513.445,60698.457,69656.499,79936.593,91733.851
2,17.2,45.9,69.3,295.84,789.48,1191.96,2106.81,3180.87,4802.49,5.088448e+03,13579.056,20501.712,36237.132,54710.964,82602.828,96702.579,146001.933,220434.291,332812.557
3,151.5,41.3,58.5,22952.25,6256.95,8862.75,1705.69,2416.05,3422.25,3.477266e+06,947927.925,1342706.625,258412.035,366031.575,518470.875,70444.997,99782.865,141338.925,200201.625
4,180.8,10.8,58.4,32688.64,1952.64,10558.72,116.64,630.72,3410.56,5.910106e+06,353037.312,1909016.576,21088.512,114034.176,616629.248,1259.712,6811.776,36834.048,199176.704


In [13]:
poly_features_df.describe()

,TV,radio,newspaper,TV^2,TV radio,TV newspaper,radio^2,radio newspaper,newspaper^2,TV^3,TV^2 radio,TV^2 newspaper,TV radio^2,TV radio newspaper,TV newspaper^2,radio^3,radio^2 newspaper,radio newspaper^2,newspaper^3
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.00000,200.00000,2.000000e+02,2.000000e+02,2.000000e+02,200.000000,2.000000e+02,2.000000e+02,200.000000,200.000000,200.000000,2.000000e+02
mean,147.042500,23.264000,30.554000,28955.591950,3490.309900,4598.126400,760.539300,824.73275,1405.48370,6.371006e+06,7.033242e+05,9.270578e+05,115027.471030,1.249737e+05,2.155203e+05,28201.560860,29000.037585,42395.923715,8.088636e+04
std,85.854236,14.846809,21.778621,25565.429396,3360.740127,4870.716495,735.807704,937.81978,1863.38301,7.082500e+06,8.716341e+05,1.291333e+06,147815.336657,1.841252e+05,3.638612e+05,34214.831852,40818.085389,71471.522336,1.664323e+05
min,0.700000,0.000000,0.300000,0.490000,0.000000,6.090000,0.000000,0.00000,0.09000,3.430000e-01,0.000000e+00,4.263000e+00,0.000000,0.000000e+00,8.600000e+00,0.000000,0.000000,0.000000,2.700000e-02
25%,74.375000,9.975000,12.750000,5531.957500,773.445000,1119.747500,99.502500,147.93750,162.57000,4.114863e+05,6.072033e+04,8.421447e+04,7658.357000,1.461359e+04,1.575378e+04,992.574750,1154.578500,1602.270000,2.072958e+03
50%,149.750000,22.900000,25.750000,22425.065000,2069.065000,2809.675000,524.570000,416.11500,663.08500,3.358154e+06,2.822733e+05,3.975226e+05,52221.407500,4.500682e+04,8.082070e+04,12019.981000,8395.255000,9585.550000,1.707560e+04
75%,218.825000,36.525000,45.100000,47884.697500,5516.197500,6492.930000,1334.077500,1243.19000,2034.01000,1.047851e+07,1.095034e+06,1.311138e+06,182535.391000,1.621647e+05,2.404681e+05,48727.317750,42124.215000,53391.312000,9.173385e+04
max,296.400000,49.600000,114.000000,87852.960000,13540.410000,29906.760000,2460.160000,4172.40000,12996.00000,2.603962e+07,3.749340e+06,8.864364e+06,662126.049000,1.085615e+06,3.017592e+06,122023.936000,179340.750000,475653.600000,1.481544e+06


### Train | Test Split

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
# X_train, X_test, y_train, y_test = train_test_split(
#    poly_features, y, test_size=0.3, random_state=101)
## Or 
X_train, X_test, y_train, y_test = train_test_split(
    poly_features_df, y, test_size=0.3, random_state=101)

In [17]:
X_train.head()

,TV,radio,newspaper,TV^2,TV radio,TV newspaper,radio^2,radio newspaper,newspaper^2,TV^3,TV^2 radio,TV^2 newspaper,TV radio^2,TV radio newspaper,TV newspaper^2,radio^3,radio^2 newspaper,radio newspaper^2,newspaper^3
85,193.2,18.4,65.7,37326.24,3554.88,12693.24,338.56,1208.88,4316.49,7.211430e+06,686802.816,2452333.968,65409.792,233555.616,833945.868,6229.504,22243.392,79423.416,283593.393
183,287.6,43.0,71.8,82713.76,12366.80,20649.68,1849.00,3087.40,5155.24,2.378848e+07,3556691.680,5938847.968,531772.400,887936.240,1482647.024,79507.000,132758.200,221675.320,370146.232
127,80.2,0.0,9.2,6432.04,0.00,737.84,0.00,0.00,84.64,5.158496e+05,0.000,59174.768,0.000,0.000,6788.128,0.000,0.000,0.000,778.688
53,182.6,46.2,58.7,33342.76,8436.12,10718.62,2134.44,2711.94,3445.69,6.088388e+06,1540435.512,1957220.012,389748.744,495200.244,629182.994,98611.128,125291.628,159190.878,202262.003
100,222.4,4.3,49.8,49461.76,956.32,11075.52,18.49,214.14,2480.04,1.100030e+07,212685.568,2463195.648,4112.176,47624.736,551560.896,79.507,920.802,10664.172,123505.992


----
----

## Scaling the Data

While our particular data set has all the values in the same order of magnitude ($1000s of dollars spent), typically that won't be the case on a dataset, and since the mathematics behind regularized models will sum coefficients together, its important to standardize the features. Review the theory videos for more info, as well as a discussion on why we only **fit** to the training data, and **transform** on both sets separately.

In [18]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# help(StandardScaler)

In [19]:
scaler = StandardScaler() # mean is 0 and std is 1

In [20]:
scaler.fit(X_train)  # Fit only with training data

StandardScaler()

In [21]:
X_train = scaler.transform(X_train)

In [27]:
pd.DataFrame(X_train,columns=polynomial_converter.get_feature_names_out()).describe()

,TV,radio,newspaper,TV^2,TV radio,TV newspaper,radio^2,radio newspaper,newspaper^2,TV^3,TV^2 radio,TV^2 newspaper,TV radio^2,TV radio newspaper,TV newspaper^2,radio^3,radio^2 newspaper,radio newspaper^2,newspaper^3
count,1.400000e+02,1.400000e+02,1.400000e+02,1.400000e+02,1.400000e+02,1.400000e+02,1.400000e+02,1.400000e+02,1.400000e+02,1.400000e+02,1.400000e+02,1.400000e+02,1.400000e+02,1.400000e+02,1.400000e+02,1.400000e+02,1.400000e+02,1.400000e+02,1.400000e+02
mean,-5.963484e-16,7.295751e-16,8.596298e-16,-1.903239e-16,1.998401e-16,-3.806479e-16,-7.454355e-16,1.776357e-16,4.948423e-16,-1.586033e-16,4.314009e-16,-2.537653e-16,4.123686e-17,3.489272e-16,1.205385e-16,3.520993e-16,-2.577303e-16,1.919100e-16,1.903239e-17
std,1.003591e+00,1.003591e+00,1.003591e+00,1.003591e+00,1.003591e+00,1.003591e+00,1.003591e+00,1.003591e+00,1.003591e+00,1.003591e+00,1.003591e+00,1.003591e+00,1.003591e+00,1.003591e+00,1.003591e+00,1.003591e+00,1.003591e+00,1.003591e+00,1.003591e+00
min,-1.791651e+00,-1.587900e+00,-1.406210e+00,-1.183328e+00,-1.100296e+00,-9.503267e-01,-1.058262e+00,-9.014749e-01,-7.955194e-01,-9.339172e-01,-8.541183e-01,-7.239764e-01,-8.205075e-01,-6.910813e-01,-5.897670e-01,-8.480498e-01,-7.263494e-01,-6.159394e-01,-5.465953e-01
25%,-8.935153e-01,-9.418782e-01,-8.188434e-01,-9.540267e-01,-8.321267e-01,-7.276434e-01,-9.328183e-01,-7.354491e-01,-6.931105e-01,-8.712443e-01,-7.508566e-01,-6.540714e-01,-7.603748e-01,-6.061558e-01,-5.462760e-01,-8.220822e-01,-6.932928e-01,-5.856280e-01,-5.296475e-01
50%,6.811570e-02,2.291474e-02,-2.431541e-01,-2.079991e-01,-2.475210e-01,-3.648778e-01,-2.784624e-01,-4.368949e-01,-4.215194e-01,-3.830247e-01,-4.447914e-01,-3.910945e-01,-4.280434e-01,-4.455220e-01,-3.795768e-01,-4.456674e-01,-4.975303e-01,-4.634604e-01,-4.290904e-01
75%,8.208642e-01,8.927545e-01,6.349765e-01,7.337507e-01,5.979052e-01,3.097657e-01,7.910605e-01,4.475021e-01,3.197085e-01,5.810136e-01,4.217087e-01,2.185625e-01,4.370843e-01,1.540810e-01,1.210100e-01,6.214368e-01,2.892558e-01,1.679267e-01,5.768634e-02
max,1.717813e+00,1.776159e+00,3.260026e+00,2.270450e+00,2.638076e+00,4.816926e+00,2.342701e+00,3.358167e+00,4.887790e+00,2.729282e+00,3.289794e+00,5.562729e+00,3.202477e+00,4.959565e+00,6.926308e+00,2.816674e+00,3.810884e+00,4.795065e+00,6.398530e+00


In [28]:
X_test = scaler.transform(X_test)

In [29]:
X_train.mean()

np.float64(7.47939721852737e-17)

In [30]:
X_train.std()

np.float64(1.0)

In [31]:
X_test.mean()

np.float64(-0.09036461660617744)

In [32]:
X_test.std()

np.float64(1.0155973297593028)

## Ridge Regression

Make sure to view video lectures for full explanation of Ridge Regression and choosing an alpha.

In [33]:
from sklearn.linear_model import Ridge

In [34]:
ridge_model = Ridge(alpha=10)

In [35]:
ridge_model.fit(X_train,y_train)

Ridge(alpha=10)

In [36]:
test_predictions = ridge_model.predict(X_test)

In [37]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [38]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [39]:
MAE

0.5774404204714166

In [40]:
RMSE

np.float64(0.8946386461319648)

How did it perform on the training set? (This will be used later on for comparison)

In [42]:
# Training Set Performance
train_predictions = ridge_model.predict(X_train)
MAE_train = mean_absolute_error(y_train,train_predictions)
MSE_train = mean_squared_error(y_train,train_predictions)
RMSE_train = np.sqrt(MSE_train)
print("MAE_train",MAE_train)
print("MSE_train",MSE_train)
print("RMSE_train",RMSE_train)

MAE_train 0.5288348183025304
MSE_train 0.7211075569495061
RMSE_train 0.8491805208255228


### Choosing an alpha value with Cross-Validation

Review the theory video for full details.

In [43]:
from sklearn.linear_model import RidgeCV

In [ ]:
# help(RidgeCV)

In [60]:
# Choosing a scoring: https://scikit-learn.org/stable/modules/model_evaluation.html
# Negative RMSE so all metrics follow convention "Higher is better"

# See all options: sklearn.metrics.SCORERS.keys()
ridge_cv_model = RidgeCV(alphas=(0.005,0.009,0.01,0.1,0.2,0.5,1.0, 10.0),scoring='neg_mean_absolute_error')

In [61]:
# The more alpha options you pass, the longer this will take.
# Fortunately our data set is still pretty small
ridge_cv_model.fit(X_train,y_train)

RidgeCV(alphas=(0.005, 0.009, 0.01, 0.1, 0.2, 0.5, 1.0, 10.0),
        scoring='neg_mean_absolute_error')

In [62]:
ridge_cv_model.alpha_

np.float64(0.01)

In [53]:
test_predictions = ridge_cv_model.predict(X_test)

In [54]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)
print("MAE",MAE)
print("MSE",MSE)
print("RMSE",RMSE)

MAE 0.4101873183288653
MSE 0.33984362776937377
RMSE 0.5829610859820523


In [63]:
# Training Set Performance
train_predictions = ridge_cv_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MSE = mean_squared_error(y_train,train_predictions)
RMSE = np.sqrt(MSE)
print("MAE",MAE)
print("MSE",MSE)
print("RMSE",RMSE)

MAE 0.28912714580437177
MSE 0.1890609900921818
RMSE 0.43481144199777194


In [64]:
ridge_cv_model.coef_

array([ 6.9038259 ,  0.49047734,  0.29464737, -9.7103904 ,  5.06456503,
       -1.66613758, -1.30427348,  0.76860722,  0.35722042,  4.58992377,
       -1.52624141,  1.42375451,  0.43356846, -0.31710017,  0.03883517,
        0.71412093, -0.3097189 , -0.21689709, -0.24610412])

In [67]:
# ridge_cv_model.feature_names_in_

In [ ]:
ridge_cv_model.intercept_

np.float64(14.311428571428573)


-----

## Lasso Regression

In [68]:
from sklearn.linear_model import LassoCV

In [82]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html
lasso_cv_model = LassoCV(eps=0.1,n_alphas=100,cv=5)

In [83]:
lasso_cv_model.fit(X_train,y_train)

LassoCV(cv=5, eps=0.1)

In [85]:
lasso_cv_model.alpha_

np.float64(0.4943070909225831)

In [86]:
test_predictions = lasso_cv_model.predict(X_test)

In [88]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)
print(RMSE)

1.1308001022762548


In [89]:
# Training Set Performance
# Training Set Performance
train_predictions = lasso_cv_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.691280714082071

In [90]:
lasso_cv_model.coef_

array([1.002651  , 0.        , 0.        , 0.        , 3.79745279,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])

In [91]:
lasso_cv_model.intercept_

np.float64(14.311428571428573)

## Elastic Net

Elastic Net combines the penalties of ridge regression and lasso in an attempt to get the best of both worlds!

In [92]:
from sklearn.linear_model import ElasticNetCV

In [93]:
elastic_model = ElasticNetCV(l1_ratio=[.1, .5, .7,.9, .95, .99, 1],tol=0.01)

In [95]:
elastic_model.fit(X_train,y_train)

ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], tol=0.01)

In [96]:
elastic_model.l1_ratio_

np.float64(1.0)

In [97]:
test_predictions = elastic_model.predict(X_test)

In [99]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)
print(RMSE)

0.7485546215633726


In [100]:
# Training Set Performance
# Training Set Performance
train_predictions = elastic_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MSE = mean_squared_error(y_train,train_predictions)
RMSE = np.sqrt(MSE)
print(RMSE)

0.6441168601816856


In [101]:
elastic_model.coef_

array([ 3.78993643,  0.89232919,  0.28765395, -1.01843566,  2.15516144,
       -0.3567547 , -0.271502  ,  0.09741081,  0.        , -1.05563151,
        0.2362506 ,  0.07980911,  1.26170778,  0.01464706,  0.00462336,
       -0.39986069,  0.        ,  0.        , -0.05343757])

# Normalization

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer

# Loading Data
df = pd.read_csv("Advertising.csv")

# Preparing input (X) and output (y)
X = df.drop('sales',axis=1)
y = df['sales']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=101)

print(np.array(X_train).min())
print(np.array(X_train).max())

print(np.array(X_test).min())
print(np.array(X_test).max())

# Applying Normalizer
normalizer = Normalizer()
normalizer.fit(X_train)
X_train = normalizer.transform(X_train)
X_test = normalizer.transform(X_test)

print("-----------")
print(X_train.min())
print(X_train.max())

print(X_test.min())
print(X_test.max())


-----
---